# Tutorial 9. Relational datasets (with star scheme)

![](https://raw.githubusercontent.com/sb-ai-lab/LightAutoML/39cb56feae6766464d39dd2349480b97099d2535/imgs/LightAutoML_logo_big.png)



In this tutorial, we will look at how to use LightAutoML with relational datasets.

Official LightAutoML github repository is [here](https://github.com/AILab-MLTools/LightAutoML)

### Install LightAutoML

Run this cell to install the latest version of LightAutoML.

In [1]:
#! pip install -U lightautoml

### Import necessary libraries

In [1]:
# Standard python libraries
import os
from os.path import join as pjoin
import time
import pickle

# ML and DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split

# Imports from lightautoml package
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM

from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.reader.base import DictToPandasSeqReader
from lightautoml.tasks import Task

# Import Feature Generator Transformer
from lightautoml.pipelines.features.generator_pipeline import FeatureGeneratorPipeline

### Relational data

Consider data that is a set of linked tables. Usually in this case there is a separate main table containing the objects identifiers and the corresponding values ​​of the target variable, as well as possibly the values ​​of other features (so called fact table). Other tables contain additional or auxiliary information, for example, records about all customer transactions (there can be an arbitrary number for a user with a specific identifier etc), the correspondence between the values ​​of one feature and the values ​​of another (the correspondence between an employee's department and his salary, for example), etc (so called dimension tables). However the organization of the data may differ from this scheme. To apply machine learning algorithms and LightAutoML, it is necessary to create a single dataset with all the features for each of the objects. For this we need to set the correspondence between the columns of the main and auxiliary tables for the correct aggregation of features. Such tables can form different schemas. 

In this example, we will consider one of the simplest and most common schemes for organizing tables - the so-called star scheme, in which there is one main table, and there are connections only between the main and auxiliary tables by specified columns, but not between separate auxiliary tables, not sequentially, etc. At the present moment, this is the only scheme supported in LightAutoML, support for more complex schemes is in development. Note that the connection between the main and each auxiliary table is carried out by a single key, but they may differ for different tables. Also, the columns for binding must be the primary key.

Consider an example of data with a star scheme organization. The dataset contains data on the sale of meals in the restaurant chain, consists of three tables: the main one containing information about completed orders (`train` and `test` parts), and two auxiliary tables containing information about restaurants (`fulfilment_center_info`) and available dishes (`meal_info`). The tables and the scheme of their organization are shown in the image below.

![](https://raw.githubusercontent.com/sb-ai-lab/LightAutoML/master/imgs/Star_scheme_tables.png)

Load the data:

In [2]:
fulfilment_center_info = pd.read_csv('./data/fulfilment_center_info.csv')
meal_info = pd.read_csv('./data/meal_info.csv')
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

For the convenience of further use, we will save datasets and paths to them in dictionaries.

In [3]:
data_dir = 'data'

filepaths = {
    'fulfilment_center_info': pjoin(data_dir, 'fulfilment_center_info.csv'),
    'meal_info': pjoin(data_dir, 'meal_info.csv'),
    'train': pjoin(data_dir, 'train.csv'),
    'test': pjoin(data_dir, 'test.csv'),
}

In [4]:
dataframes = {}
for df_name in filepaths.keys():
    dataframes[df_name] = pd.read_csv(filepaths[df_name])
    print(df_name, dataframes[df_name].shape)

fulfilment_center_info (77, 5)
meal_info (51, 3)
train (456548, 9)
test (32573, 8)


### Create sequential star scheme dictionary

For further use of LightAutoML, you need to specify the data schema. It is necessary to specify secondary tables in the dictionary as the key to which the dictionary of the remaining parameters corresponds. The following parameters are specified in this dictionary:

- `'case'` -  the type of column that plays the role of a key for binding. If `'ids'`, then the column is treated as a set of unique identifiers (ids), and if `'next_values'`, then it is treated as a set of timestamps.

- `'params'` - dictionary of timestamp processing and interpretation parameters in case of linking by `'next_values'` type column. In case of `'ids'` it might be set empty.

- `'scheme'` -  dictionary describing the scheme of relationship between the main and secondary table. Consists of the next keys: `'to'` - the name of the table, the relationship with which is being considered (in case of star scheme, the name `'plain'` should be specified here); `'from_id'` - the name of column for link in secondary table (from which the link exists); `'to_id'` - the name of column for link in main table (to which the link exists).

In our example, columns for linkage are IDs. Now we set a dictionary of parameters for communication taking into account the table schema:

In [5]:
seq_params = {
              'fulfilment_center_info':{'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'center_id', 'to_id': 'center_id'},
                          },
              'meal_info':{'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'meal_id', 'to_id': 'meal_id'},
                          },
              }

Create a dict with second-level tables.

In [6]:
seq_data = {'fulfilment_center_info': dataframes['fulfilment_center_info'],
       'meal_info': dataframes['meal_info']              
       }

Define train and test data samples. They must be specified in the form of a dictionary, where the main dataset is specified by the `'plain'` key, and the dictionary with secondary tables is specified by the `'seq'` key (like the `seq_data` dictionary). Note that train and test data differ only in plain data, and train plain data must contain a column with the target variable.

In [7]:
X_train = {'plain':dataframes['train'] , 
           'seq': seq_data
          }

X_test = {'plain':dataframes['test'] , 
          'seq': seq_data
          }

### Create Task snd Sequential Reader for the star scheme data

To work with linked tables in LightAutoML, it is not possible to use tabular presets like `TabularAutoML`, so we have to set all the pipeline manually. You can see more details about creating custom pipelines in [this tutorial](https://github.com/sb-ai-lab/LightAutoML/blob/master/examples/tutorials/Tutorial_6_custom_pipeline.ipynb).

First we will set task and roles for our objective. Than it is necessary to create `DictToPandasSeqReader` to process data in form of relational tables. It requires setting the task and sequential data parameters dict as arguments (more details about this reader you can see [here](https://github.com/sb-ai-lab/LightAutoML/blob/master/lightautoml/reader/base.py#L651)):

In [8]:
task = Task('reg', metric='mae')
roles={'target': 'num_orders'}
reader = DictToPandasSeqReader(task=task, seq_params=seq_params)

### Create Feature Generator Pipeline

In addition to aggregating data from all related tables into one, LightAutoML has the ability to perform additional feature generation by using `FeatureGeneratorPipeline`. Features can be generated using various aggregations (taking the average, median, counting unique values, etc.), extracting date features (year, day, difference between dates, weekend or weekday, etc.), different transformations, as well as using so-called interesting values, that is, constructing features by objects with a certain value of a set of categorical features (conditional feature generation, like "where" clause). For aggregation and transformation LightAutoML uses according primitives from FeatureTools, detailed info is available [here](https://docs.featuretools.com/en/v0.16.0/automated_feature_engineering/primitives.html).

Define interesing values parameters for feature generation in corresponding tables.

In [9]:
interesting_values = {
    'fulfilment_center_info': {'center_type': ['TYPE_A', 'TYPE_C'], 'city_code': [647, 456, 703]},
    'meal_info': {'category': ['Extras', 'Seafood'], 'cuisine': ['Continental', 'Thai']}
}

So, in our example we want to generate features by orders where `'center_type'` feature was equal to `'TYPE_A'` or `'TYPE_C'`, and `'city_code'` feature was equal to `647`, `456` or `703`, and similarly for features of ordered meal from `meal_info` table. 

Params of feature generator:
- seq_params: secondary tables or sequence related parameters.
- max_gener_features: maximum number of generated features.
- max_depth: maximum allowed depth of features (that is, the number of consecutively applied aggregation and transformation primitives in a superposition to obtain features).
- agg_primitives: list of aggregation primitives. By default it is \[`"entropy"`, `"count"`, `"mean"`, `"std"`, `"median"`, `"max"`, `"sum"`, `"num_unique"`, `"min"`, `"percent_true"`\].
- trans_primitives: list of transform primitives. By default it is \[`"hour"`, `"month"`, `"weekday"`, `"is_weekend"`, `"day"`, `"time_since_previous"`, `"week"`, `"age"`, `"time_since"`\].
- interesting_values: categorical values if the form of {'table_name': {'column': [values]}} for feature generation in corresponding slices (like the `interesting_values` dictionary above).
- generate_interesting_values: whether generate feature in slices of unique categories or not.
- per_top_categories: percent of most frequent categories for feature generation in corresponding slices. If number of unique values is less than 10, then the all values are be used. 
- sample_size: size of data to make generated feature selection on it.
- n_jobs: number of processes to run in parallel

More details about `FeatureGeneratorPipeline` are available [here](https://github.com/sb-ai-lab/LightAutoML/blob/53cd4a7634099eb8815dd5d9f0dda5eaae51c3c2/lightautoml/pipelines/features/generator_pipeline.py#L21).

In [10]:
generator = FeatureGeneratorPipeline(seq_params,
                                     max_gener_features=500,
                                     interesting_values = interesting_values,
                                     generate_interesting_values = True,
                                     per_top_categories = 25,
                                     sample_size = None,
                                     n_jobs = 16)


### Create one-level ML pipeline for AutoML

Next we will compose the entire pipeline. We will add the basic simplest transformations to the pipeline of feature generation (encoding categorical features, converting date features to appropriate format, defining numeric types, defining roles). The set of algorithms will consist only of LightGBM gradient boosting, and no pre-selection or post-selection of features will be used.

In [11]:
simpletransf = LGBSimpleFeatures()
feats = generator.append(simpletransf)

model = BoostLGBM()

pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats,post_selection=None)

Initialize `AutoML` instance:

In [12]:
automl = AutoML(reader, [[pipeline_lvl1],], skip_conn=False)

### Train AutoML on loaded data

Let's train our model on train data and look at the logs of training. For more detailed info we will set verbosity level to 3:

In [13]:
%%time

train_pred = automl.fit_predict(X_train, roles=roles, verbose=3)

[15:52:16] Feats was rejected during automatic roles guess: ['id']
[15:52:16] Layer 1 train process start. Time left 9999999997.67 secs
[15:52:16] This selector only for holdout training. fit_on_holout argument added just to be compatible
[15:52:16] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
EntitySet scattered to 16 workers in 4 seconds


/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3

[15:52:27] Training until validation scores don't improve for 100 rounds
[15:52:31] [100]	valid's l1: 203.63
[15:52:35] [200]	valid's l1: 203.659
[15:52:36] Early stopping, best iteration is:
[136]	valid's l1: 203.599
[15:52:37] LightGBM fitting and predicting completed
[15:52:37] Started iteration 0, chunk = ['ft__plain_center_id.MAX(fulfilment_center_info.op_area)', 'ft__plain_meal_id.COUNT(meal_info WHERE cuisine = Italian)', 'ft__plain_meal_id.COUNT(meal_info WHERE cuisine = Continental)', 'ft__plain_center_id.MAX(fulfilment_center_info.city_code WHERE center_type = TYPE_A)', 'ft__plain_center_id.MAX(fulfilment_center_info.op_area WHERE center_type = TYPE_B)', 'ft__plain_center_id.MEDIAN(fulfilment_center_info.op_area)', 'ft__plain_meal_id.COUNT(meal_info WHERE cuisine = Indian)', 'ft__plain_center_id.MEAN(fulfilment_center_info.op_area)', 'ft__plain_center_id.MAX(fulfilment_center_info.city_code)', 'ft__plain_center_id.MAX(fulfilment_center_info.op_area WHERE center_type = TYPE_A)

/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3

[15:53:47] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[15:53:47] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:53:48] Training until validation scores don't improve for 100 rounds
[15:53:50] [100]	valid's l1: 85.4464
[15:53:52] [200]	valid's l1: 77.5188
[15:53:54] [300]	valid's l1: 73.9796
[15:53:55] [400]	valid's l1: 71.6819
[15:53:57] [500]	valid's l1: 69.9947
[15:53:58] [600]	valid's l1: 68.7763
[15:54:00] [700]	valid's l1: 67.7473
[15:54:01] [800]	valid's l1: 66.9755
[15:54:02] [900]	valid's l1: 66.3252
[15:54:04] [1000]	valid's l1: 65.803
[15:54:05] [1100]	valid's l1: 65.3836
[15:54:07] [1200]	valid's l1: 64.9446
[15:54:08] [1300]	valid's l1: 64.5974
[15:54:09] [1400]	valid's l1: 64.3065
[15:54:11] [1500]	valid's l1: 63.9985
[15:54:12] [1600]	valid's l1: 63.7705
[15:54:13] [1700]	valid's l1: 63.5289
[15:54:15] [1800]	valid's l1: 63.3132
[15:54:16] [1900]	valid's l1: 63.1277
[15:54:18] [2000]	valid's l1: 62.9634
[15:54:19] [2100]	valid's l1: 62.8097
[

In the **"Finally selected feats"** line, we can see the features generated by `FeatureGenerationPipeline` and selected using LightGBM, obtained using aggregations, tarnsformations and interesting values. For example, `'ft__plain_center_id.MEDIAN(fulfilment_center_info.region_code WHERE center_type = TYPE_A)'` feature is median over `'region_code'` column in `fulfilment_center_info` table (which linked with `'plain'` dataset by `'center_id'` key) where `'center_type'` value equals `'TYPE_A'`.

### Analyze fitted model

Let's see the generated features and their importances (received from LightGBM) which we get as a result of training the model:

In [14]:
feature_imps = model.get_features_score()
feature_imps

ord__checkout_price                                                                          7.374265e+10
meal_id                                                                                      6.690076e+10
ord__base_price                                                                              5.716648e+10
ft__plain_center_id.MAX(fulfilment_center_info.op_area)                                      3.194187e+10
week                                                                                         3.132576e+10
homepage_featured                                                                            3.118942e+10
emailer_for_promotion                                                                        2.311121e+10
center_id                                                                                    1.345948e+10
ft__plain_meal_id.COUNT(meal_info WHERE cuisine = Italian)                                   1.170391e+10
ft__plain_meal_id.COUNT(meal_info WHERE cuisin

Quite a large number of features heve non-zero importances:

In [15]:
feature_imps.index[feature_imps > 0]

Index(['ord__checkout_price', 'meal_id', 'ord__base_price',
       'ft__plain_center_id.MAX(fulfilment_center_info.op_area)', 'week',
       'homepage_featured', 'emailer_for_promotion', 'center_id',
       'ft__plain_meal_id.COUNT(meal_info WHERE cuisine = Italian)',
       'ft__plain_meal_id.COUNT(meal_info WHERE cuisine = Indian)',
       'ft__plain_meal_id.COUNT(meal_info WHERE cuisine = Thai)',
       'ft__plain_center_id.MAX(fulfilment_center_info.city_code WHERE center_type = TYPE_A)',
       'ft__plain_center_id.MAX(fulfilment_center_info.city_code)',
       'ft__plain_center_id.MAX(fulfilment_center_info.op_area WHERE center_type = TYPE_B)',
       'ft__plain_center_id.MAX(fulfilment_center_info.region_code)',
       'ft__plain_center_id.MEDIAN(fulfilment_center_info.op_area)',
       'ft__plain_center_id.MAX(fulfilment_center_info.op_area WHERE center_type = TYPE_A)',
       'ft__plain_center_id.MAX(fulfilment_center_info.region_code WHERE center_type = TYPE_A)',
       'ft__

Evaluating model by the out-of-fold  prediction on train data:

In [16]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(X_train['plain'][roles['target']], train_pred.data[:, 0])
print(f'MAE on train: {mae}')

MAE on train: 62.206650717961615


Making predictions for test data:

In [17]:
test_pred = automl.predict(X_test)

EntitySet scattered to 16 workers in 3 seconds


/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3

### Pickle the model (optional)

Also it is possible to save trained model as pickle file if necessary and use it further to make predictions.

In [18]:
with open('ft_model.pickle', 'wb') as f:
    pickle.dump(automl, f)
    
with open('ft_model.pickle', 'rb') as f:
    automl = pickle.load(f)
_pred = automl.predict(X_train)

EntitySet scattered to 16 workers in 4 seconds


/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,
/home/user/Miheev/venvs/miheev/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3

### Additional materials

- [Official LightAutoML github repo](https://github.com/AILab-MLTools/LightAutoML)
- [LightAutoML documentation](https://lightautoml.readthedocs.io/en/latest)
- [LightAutoML tutorials](https://github.com/AILab-MLTools/LightAutoML/tree/master/examples/tutorials)
- LightAutoML course:
    - [Part 1 - general overview](https://ods.ai/tracks/automl-course-part1) 
    - [Part 2 - LightAutoML specific applications](https://ods.ai/tracks/automl-course-part2)
    - [Part 3 - LightAutoML customization](https://ods.ai/tracks/automl-course-part3)
- [OpenDataScience AutoML benchmark leaderboard](https://ods.ai/competitions/automl-benchmark/leaderboard)